In [1]:
!pip install fastapi uvicorn pydantic scikit-learn pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.2 MB/s eta 0:00:00


In [2]:
from fastapi import FastAPI, HTTPException
import pandas as pd
import pickle
from pydantic import BaseModel
from typing import Literal
import uvicorn
from threading import Thread
import numpy as np
import requests
import json
import time
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

app = FastAPI()


with open('model.pkl', 'rb') as f:
    model = pickle.load(f)


class InsuranceInput(BaseModel):
    Pclass: int
    Age: int
    CarAge: int
    InsuranceCost: int
    DaysWithCompany: int
    HasLicence: int
    HasDamage: int

@app.get("/health")
def health():
    return {"status": "OK"}

@app.post("/predict")
async def predict_insurance(data: InsuranceInput):
    try:
        model_data = {
            "Gender": data.Pclass,
            "Driving_License": data.HasLicence,
            "Vehicle_Age": data.CarAge,
            "Annual_Premium": data.InsuranceCost,
            "Vintage": data.DaysWithCompany,
            "Age Above 38": data.Age,
            "Not_Insured and Damaged": data.HasDamage
        }

        df = pd.DataFrame([model_data])[model.feature_names_in_]

        prediction = model.predict(df)
        return {"result": "Купит страховку" if prediction == 1 else "Не купит"}

    except Exception as e:
        raise HTTPException(status_code=400, detail=str(e))

In [3]:
import threading

def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8000, reload=False)

thread = threading.Thread(target=run_server)
thread.start()

print("Сервер запущен на http://127.0.0.1:8000")

Сервер запущен на http://127.0.0.1:8000


In [4]:
data = {'Pclass': 1,
 'Age': 33,
 'CarAge': 2,
 'InsuranceCost': 5894,
 'DaysWithCompany': 288,
 'HasLicence': 1,
 'HasDamage': 1}

In [5]:
response = requests.post("http://localhost:8000/predict", json=data)
print(response.json())

INFO:     127.0.0.1:44630 - "POST /predict HTTP/1.1" 200 OK
{'result': 'Купит страховку'}


In [6]:
test_df = pd.read_csv('final_train.csv')

In [7]:
test_df.head()

,Unnamed: 0,Gender,Driving_License,Vehicle_Age,Annual_Premium,Vintage,Age Above 38,Not_Insured and Damaged,Response
0,0,1,1,1,65101.0,187,0,1,0
1,1,1,1,2,58911.0,288,1,1,1
2,2,0,1,0,38043.0,254,0,0,0
3,3,0,1,1,2630.0,76,0,1,0
4,4,0,1,1,31951.0,294,0,0,0


In [8]:
test_df = test_df.drop('Unnamed: 0', axis=1)

In [9]:
test_df_ans = test_df["Response"]

In [10]:
test_df_X = test_df.drop(['Response'],axis=1)

In [11]:
del test_df

In [12]:
def prepare_api_input(row):
    return {
        "Pclass": int(row['Gender']),
        "Age": int(row['Age Above 38']),
        "CarAge": int(row['Vehicle_Age']),
        "InsuranceCost": int(row['Annual_Premium']),
        "DaysWithCompany": int(row['Vintage']),
        "HasLicence": int(row['Driving_License']),
        "HasDamage": int(row['Not_Insured and Damaged'])
    }

In [13]:
test_cases = []
for _, row in test_df_X.iterrows():
    case = {
        "input": prepare_api_input(row)}
    test_cases.append(case)

In [14]:
data = test_cases[1]["input"]
data

{'Pclass': 1,
 'Age': 1,
 'CarAge': 2,
 'InsuranceCost': 58911,
 'DaysWithCompany': 288,
 'HasLicence': 1,
 'HasDamage': 1}

In [15]:
response = requests.post("http://localhost:8000/predict", json=data)
print(response.json())

INFO:     127.0.0.1:52642 - "POST /predict HTTP/1.1" 200 OK
{'result': 'Купит страховку'}


In [16]:
!pip install aiohttp
!pip install aiofiles

In [17]:
import aiohttp
import asyncio
import csv
from tqdm.notebook import tqdm
from asyncio import Semaphore

url = "http://localhost:8000/predict"


async def send_post_request(session, sem, item, writer, lock, input_fields):
    headers = {'Content-Type': 'application/json'}
    async with sem:
        try:
            async with session.post(url, json=item['input'], headers=headers) as response:
                response.raise_for_status()
                result_data = await response.json()
                row = {**item['input'], "result": result_data}
        except Exception as e:
            row = {**item['input'], "result": None, "error": str(e)}

        # Потокобезопасная запись в файл
        async with lock:
            writer.writerow(row)

async def process_requests_async(data, output_file="results.csv", max_concurrent=100):
    sem = Semaphore(max_concurrent)
    lock = asyncio.Lock()

    input_fields = list(data[0]["input"].keys())
    fieldnames = input_fields + ["result", "error"]


    with open(output_file, mode='w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()

        async with aiohttp.ClientSession() as session:
            tasks = [
                send_post_request(session, sem, item, writer, lock, input_fields)
                for item in data
            ]
            for f in tqdm(asyncio.as_completed(tasks), total=len(tasks)):
                await f

In [18]:
await process_requests_async(test_cases, output_file="results.csv", max_concurrent=100)

  0%|          | 0/1000 [00:00<?, ?it/s]

INFO:     127.0.0.1:57338 - "POST /predict HTTP/1.1" 200 OK
INFO:     127.0.0.1:57340 - "POST /predict HTTP/1.1" 200 OK
INFO:     127.0.0.1:57348 - "POST /predict HTTP/1.1" 200 OK
INFO:     127.0.0.1:57362 - "POST /predict HTTP/1.1" 200 OK
INFO:     127.0.0.1:57374 - "POST /predict HTTP/1.1" 200 OK
INFO:     127.0.0.1:57376 - "POST /predict HTTP/1.1" 200 OK
INFO:     127.0.0.1:57384 - "POST /predict HTTP/1.1" 200 OK
INFO:     127.0.0.1:57390 - "POST /predict HTTP/1.1" 200 OK
INFO:     127.0.0.1:57396 - "POST /predict HTTP/1.1" 200 OK
INFO:     127.0.0.1:57406 - "POST /predict HTTP/1.1" 200 OK
INFO:     127.0.0.1:57420 - "POST /predict HTTP/1.1" 200 OK
INFO:     127.0.0.1:57422 - "POST /predict HTTP/1.1" 200 OK
INFO:     127.0.0.1:57438 - "POST /predict HTTP/1.1" 200 OK
INFO:     127.0.0.1:57446 - "POST /predict HTTP/1.1" 200 OK
INFO:     127.0.0.1:57454 - "POST /predict HTTP/1.1" 200 OK
INFO:     127.0.0.1:57458 - "POST /predict HTTP/1.1" 200 OK
INFO:     127.0.0.1:57468 - "POST /predi

In [19]:
res = pd.read_csv('results.csv')
res.head()

,Pclass,Age,CarAge,InsuranceCost,DaysWithCompany,HasLicence,HasDamage,result,error
0,1,1,1,61220,115,1,1,{'result': 'Купит страховку'},NaN
1,0,1,1,32337,268,1,1,{'result': 'Купит страховку'},NaN
2,0,1,1,22492,284,1,1,{'result': 'Купит страховку'},NaN
3,0,1,1,27520,276,1,0,{'result': 'Купит страховку'},NaN
4,0,0,0,26170,117,1,0,{'result': 'Купит страховку'},NaN


In [20]:
res['result'] = np.where(res['result'] == "{'result': 'Купит страховку'}", 1, 0)